In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.initializers as KI
import keras.layers as KL
import keras.losses as KLoss
import tensorflow as tf
from keras import backend as K
from keras.engine.topology import Layer
from keras.layers import Convolution2D, GlobalAveragePooling2D, Dense, Activation, Dropout, Flatten, AveragePooling2D
from keras.models import Model
from keras.utils import conv_utils
from keras import applications
from keras import optimizers
from keras.utils import multi_gpu_model
from keras.utils.generic_utils import get_custom_objects
from keras.optimizers import Adam
import math
from keras.callbacks import LearningRateScheduler
# Seed value (can actually be different for each attribution step)
seed_value= 0
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)
# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value) # tensorflow 2.x
# tf.set_random_seed(seed_value) # tensorflow 1.x

# Utils

In [ ]:
# Swish Activation Function
def swish(x):
    return K.sigmoid(x) * x

get_custom_objects().update({"swish": Activation(swish)})


# Learning Step Decay by 10e-1 after every 4 epochs
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 4.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch) / epochs_drop))
    return lrate

# Calculates Precision Accuracy
def precision(y_true, y_pred):
    """Precision metric.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


# Calculates Recall Accuracy
def recall(y_true, y_pred):
    """Recall metric.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


# Calculates F1 score
def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# InceptionV3 with Inaturalist dataset

## Unfreezed

## Inception V3

In [ ]:
img_rows, img_cols = (334, 334)
train_batchsize = 16
val_batchsize = 16

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/train_images',
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical',
        interpolation='bilinear')
 
validation_generator = validation_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/val_images',
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,
        interpolation='bilinear')

# dimensions of our images.
img_width, img_height = 334, 334
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.load_weights('../input/inaturalist/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
for layer in base_model.layers[:len(base_model.layers)-17]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-17:]:
    layer.trainable = True
# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_2 = Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(0.0001)
model_2.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
             metrics=[precision, recall, f1, 'acc'])
lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("./inception_natural_best.h5",
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 1082
nb_validation_samples= 103
epochs=10
batch_size=16

history = model_2.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

In [ ]:
model_2.evaluate(train_generator, verbose=1)

In [ ]:
model_2.evaluate(validation_generator, verbose=1)

Inception V3 pretrained with INaturalist dataset give better result than the one on ImageNet

## Inception V3 cropped image

In [ ]:
img_rows, img_cols = (224, 224)
train_batchsize = 16
val_batchsize = 16

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/train_images_cropped',
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical',
        interpolation='bilinear')
 
validation_generator = validation_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/val_images_cropped',
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,
        interpolation='bilinear')

# dimensions of our images.
img_width, img_height = 224, 224
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.load_weights('../input/inaturalist/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
for layer in base_model.layers[:len(base_model.layers)-17]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-17:]:
    layer.trainable = True
# Add final layers
x = base_model.output
x = AveragePooling2D((4, 4), strides=(4, 4), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_3 = Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(0.0001)
model_3.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
             metrics=[precision, recall, f1, 'acc'])
lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("./inception_natural_cropping_best_test_0.h5",
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 941
nb_validation_samples= 92
epochs=10
batch_size=16

history = model_3.fit(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size,
                     verbose=1)

In [ ]:
model_3.evaluate(validation_generator)

In [ ]:
model_3.evaluate(train_generator)

# Predictions

In [ ]:
## ORIGINAL IMAGES
# dimensions of our images.
img_width, img_height = 334, 334
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.load_weights('../input/inaturalist/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
for layer in base_model.layers[:len(base_model.layers)-17]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-17:]:
    layer.trainable = True
# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_orig_predict = Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(0.0001)
model_orig_predict.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
             metrics=[precision, recall, f1, 'acc'])
model_orig_predict.load_weights('./inception_natural_best.h5')


## CROPPED IMAGES
# dimensions of our images.
img_width, img_height = 224, 224
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.load_weights('../input/inaturalist/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
for layer in base_model.layers[:len(base_model.layers)-17]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-17:]:
    layer.trainable = True
# Add final layers
x = base_model.output
x = AveragePooling2D((4, 4), strides=(4, 4), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.3)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_crop_predict = Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(0.0001)
model_crop_predict.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
             #metrics=[precision, recall, f1, 'acc'])
                metrics=[f1, 'acc'])
model_crop_predict.load_weights('./inception_crop_best.h5')

In [ ]:
# Predictions on original data
img_rows, img_cols = (334,334)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/test_images',
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode=None,
        shuffle=False,
        interpolation='bilinear')
predictions_inception_original = model_orig_predict.predict(test_generator)

# Predictions on cropped image
img_rows, img_cols = (224,224)

test_datagen_crop = ImageDataGenerator(rescale=1./255)
 
test_generator_crop = test_datagen_crop.flow_from_directory(
        '../input/bird-dasat/bird_dataset/test_images_cropped',
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode=None,
        shuffle=False,
        interpolation='bilinear')
predictions_inception_crop = model_crop_predict.predict(test_generator_crop)

# Get filenames list
filenames_orig = [x.split('/')[1][:-4] for x in test_generator.filenames]
filenames_crop = [x.split('/')[1][:-4] for x in test_generator_crop.filenames]

## Predictions with Max Probability between the two models

In [ ]:
category = []
all_probas = []
for idx, elem in enumerate(filenames_orig):
    inception_orig = (np.max(predictions_inception_original[idx]), np.argmax(predictions_inception_original[idx]))
    try:
        indice = filenames_crop.index(elem)
        inception_crop = (np.max(predictions_inception_crop[indice]), np.argmax(predictions_inception_crop[indice]))
        all_proba = [inception_orig, inception_crop]
    except ValueError:
        all_proba = [inception_orig]
    max_proba, cat = all_proba[0]
    for prob, label in all_proba[1:]:
        if prob > max_proba:
            max_proba = prob
            cat = label
    all_probas.append(all_proba)
    category.append(cat)

In [ ]:
for lst in all_probas:
    if len(lst) > 1:
        if lst[0][1] != lst[1][1]:
            print(lst)

In [ ]:
df = pd.DataFrame({'Id': filenames_orig, 'Category': category})
df.head()

In [ ]:
df.to_csv('./submissions_2.csv', index=False)

0.87741

## Predictions with only inceptionv3orig on original image & inceptionv3crop on cropped images with condition

if the difference between proba on original image and proba on cropped image is less than 0.15, I will take the prediction on original image (as the inceptionv3orig performs better than the inceptionv3crop)

In [ ]:
category = []
all_probas = []
thresh = 0.15
for idx, elem in enumerate(filenames_orig):
    inception_orig = [np.max(predictions_inception_original[idx]), np.argmax(predictions_inception_original[idx])]
    try:
        indice = filenames_crop.index(elem)
        inception_crop = [np.max(predictions_inception_crop[indice]), np.argmax(predictions_inception_crop[indice])]
        if abs(inception_orig[0] - inception_crop[0]) < thresh:
            inception_crop[0] = 0.
        all_proba = [inception_orig, inception_crop]
    except ValueError:
        all_proba = [inception_orig]
    max_proba, cat = all_proba[0]
    for prob, label in all_proba[1:]:
        if prob > max_proba:
            max_proba = prob
            cat = label
    all_probas.append(all_proba)
    category.append(cat)
        

In [ ]:
df_ = pd.DataFrame({'Id': filenames_orig, 'Category': category})
df_.to_csv('./submissions_3.csv', index=False)

0.88387